In [ ]:
%pip install llama-index-llms-openai
!pip install llama-index

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.ERROR)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


In [ ]:
import os
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Configure Ollama LLM
ollama_llm = Ollama(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding

In [ ]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()
# documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()

In [ ]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents)


In [ ]:
# Configure the chat engine in ReAct mode
chat_engine = vector_index.as_chat_engine(
    chat_mode="react",
    llm=ollama_llm,
    verbose=True
)


In [ ]:
response = chat_engine.chat(
    "Use the tool to answer what did Paul Graham do in the summer of 1995?"
)
print("Chat Response:")
print(response)


In [ ]:
response = chat_engine.chat("What did I ask you before?")
print("Follow-Up Response:")
print(response)


In [ ]:
chat_engine.reset()
response = chat_engine.chat("What did I ask you before?")
print("After Reset Response:")
print(response)


In [ ]:
streaming_response = chat_engine.stream_chat("Summarize the document.")
print("Streaming Response:")
for token in streaming_response.response_gen:
    print(token, end="")


In [ ]:
%pip install llama-index-llms-anthropic


In [ ]:
from llama_index.llms.anthropic import Anthropic

# Switch to Anthropic's Claude model
anthropic_llm = Anthropic(model="claude-2")
chat_engine = vector_index.as_chat_engine(
    llm=anthropic_llm,
    chat_mode="react",
    verbose=True
)
